# Independent fission yield sample production

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sandy

### Retrieve nuclear data information

In [3]:
e = 0.0253
zam = 922350
tape = sandy.get_endf6_file('jeff_33','nfpy', zam)

In [4]:
tape_rdd = sandy.get_endf6_file("jeff_33", "decay", "all")
decay_data = sandy.DecayData.from_endf6(tape_rdd)

In [5]:
nfpy = sandy.Fy.from_endf6(tape)
nfpy.data.set_index(["MAT", "MT", "ZAM", "ZAP"]).head()

E          FY         DFY
MAT  MT  ZAM    ZAP                                      
9228 454 922350 10010 2.53000e-02 1.71100e-05 2.94830e-06
                10020 2.53000e-02 8.40000e-06 2.43890e-06
                10030 2.53000e-02 1.08000e-04 5.86350e-06
                20030 2.53000e-02 0.00000e+00 0.00000e+00
                20040 2.53000e-02 1.70000e-03 8.13470e-05

### Obtain covariance matrix

Assuming the uncertaities equal to the standard deviations, it is possible to build the diagonal covariance matrix as follows:

In [6]:
fy_stdev = nfpy.data.query(f"ZAM=={zam} & E=={e} & MT==454").set_index('ZAP').DFY
cov = sandy.CategoryCov.from_stdev(fy_stdev)
cov.data.head().T.head()

ZAP,10010,10020,10030,20030,20040
ZAP,,,,,
10010,8.69247e-12,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00
10020,0.00000e+00,5.94823e-12,0.00000e+00,0.00000e+00,0.00000e+00
10030,0.00000e+00,0.00000e+00,3.43806e-11,0.00000e+00,0.00000e+00
20030,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00
20040,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,6.61733e-09


This procedure handles relative covariance matrices. SANDY can work with relative or absolute covariance matrices thanks to the `relative` kwarg option inserted in `sandy.CategoryCov.sampling()` method. To relativize the covariance matrix it is possible to use the so called "sandwich rule" (implemented in `sandy.CategoryCov.sandwich()`), where the diagonal sensitivity matrix $S$ will be equal to the collection of the best estimates of the independent fission yields.

In [7]:
S = np.diag(1 / nfpy.data.query(f"ZAM=={zam} & E=={e} & MT==454").set_index("ZAP").FY)
S[S == np.inf] = 0

In [8]:
idx = cov.data.index
cov_relative = sandy.CategoryCov(pd.DataFrame(cov.sandwich(S).data.values, index=idx, columns=idx))
cov_relative.data.head().T.head()

ZAP,10010,10020,10030,20030,20040
ZAP,,,,,
10010,2.96923e-02,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00
10020,0.00000e+00,8.43004e-02,0.00000e+00,0.00000e+00,0.00000e+00
10030,0.00000e+00,0.00000e+00,2.94759e-03,0.00000e+00,0.00000e+00
20030,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00
20040,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,2.28974e-03


### Obtain perturbation coefficients

In [9]:
nsmp = 500
coeff = cov_relative.sampling(nsmp)